# OpenAI Tools

These output parsers extract tool calls from OpenAI's function calling API responses. This means they are only usable with models that support function calling, and specifically the latest `tools` and `tool_choice` parameters. We recommend familiarizing yourself with [function calling](/docs/modules/model_io/chat/function_calling) before reading this guide.

There are a few different variants of output parsers:

- [JsonOutputToolsParser](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.openai_tools.JsonOutputToolsParser.html#langchain.output_parsers.openai_tools.JsonOutputToolsParser): Returns the arguments of the function call as JSON
- [JsonOutputKeyToolsParser](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.openai_tools.JsonOutputKeyToolsParser.html#langchain.output_parsers.openai_tools.JsonOutputKeyToolsParser): Returns the value of specific key in the function call as JSON
- [PydanticToolsParser](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.openai_tools.PydanticToolsParser.html#langchain.output_parsers.openai_tools.PydanticToolsParser): Returns the arguments of the function call as a Pydantic Model

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [2]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])

In [4]:
model.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Joke',
   'description': 'Joke to tell user.',
   'parameters': {'type': 'object',
    'properties': {'setup': {'description': 'question to set up a joke',
      'type': 'string'},
     'punchline': {'description': 'answer to resolve the joke',
      'type': 'string'}},
    'required': ['setup', 'punchline']}}}]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

## JsonOutputToolsParser

In [6]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

In [7]:
parser = JsonOutputToolsParser()

In [9]:
chain = prompt | model | parser

In [10]:
chain.invoke({"input": "tell me a joke"})

[{'type': 'Joke',
  'args': {'setup': "Why don't scientists trust atoms?",
   'punchline': 'Because they make up everything!'}}]

To include the tool call id we can specify `return_id=True`:

In [13]:
parser = JsonOutputToolsParser(return_id=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})

[{'type': 'Joke',
  'args': {'setup': "Why don't scientists trust atoms?",
   'punchline': 'Because they make up everything!'},
  'id': 'call_Isuoh0RTeQzzOKGg5QlQ7UqI'}]

## JsonOutputKeyToolsParser

This merely extracts a single key from the returned response. This is useful for when you are passing in a single tool and just want it's arguments.

In [14]:
from typing import List

from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

In [19]:
parser = JsonOutputKeyToolsParser(key_name="Joke")

In [20]:
chain = prompt | model | parser

In [23]:
chain.invoke({"input": "tell me a joke"})

[{'setup': "Why don't scientists trust atoms?",
  'punchline': 'Because they make up everything!'}]

Certain models can return multiple tool invocations each call, so by default the output is a list. If we just want to return the first tool invocation, we can specify `return_single=True`

In [24]:
parser = JsonOutputKeyToolsParser(key_name="Joke", return_single=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

## PydanticToolsParser

This builds on top of `JsonOutputToolsParser` but passes the results to a Pydantic Model. This allows for further validation should you choose.

In [25]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

In [28]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticToolsParser(tools=[Joke])

In [30]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])
chain = prompt | model | parser

In [31]:
chain.invoke({"input": "tell me a joke"})

[Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')]